Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.io.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

In [ ]:
dg = DataGroup('cage.json', load=True)
run = 82
str_query = f'run=={run} and skip==False'
dg.file_keys.query(str_query, inplace=True)
# view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
# print(dg.file_keys[view_cols])
print(dg.file_keys)

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.file_keys['runtype'])
runtype = runtype_list[0]

rt_min = dg.file_keys['runtime'].sum()
u_start = dg.file_keys.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str
print(rt_min)

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['angle'])[0]
    angle_det = 270 + angle
    print(f'Radius: {radius}; Angle: {angle}')
    
else:
    radius = 'n/a'
    angle = 'n/a'
    angle_det = 'n/a'

In [ ]:
lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
hit_list = lh5_dir + dg.file_keys['hit_path'] + '/' + dg.file_keys['hit_file']
print(hit_list)
df_hit = lh5.load_dfs(hit_list, ['energy', 'trapEmax', 'trapEmax_cal', 'trapEftp', 'timestamp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [ ]:
print(len(df_hit['timestamp']))
print(len(df_hit['trapEmax']))
ts_array  = np.array(df_hit['ts_sec'])
ts_max = np.amax(ts_array)
df_hit['timestamp'].min

In [ ]:
etype = 'trapEmax'
# etype = 'trapEmax_cal'
# etype = 'trapEftp'

In [ ]:
# Baseline vs E



elo, ehi, epb = 0, 25000, 100
tlo, thi, tpb = 0, 2000, 1
# tlo, thi, tpb = 0, 1800, 1
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_hit['ts_sec'], df_hit['trapEmax'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='viridis', norm=LogNorm(1,100))

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (10s sec)', ha='right', x=1)
plt.ylabel('trapEmax', ha='right', y=1)
plt.title(f'Energy vs time, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E



elo, ehi, epb = 0, 500, 5
tlo, thi, tpb = 0, 2000, 1
# tlo, thi, tpb = 0, 1800, 1
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_hit['ts_sec'], df_hit['trapEmax_cal'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='viridis', norm=LogNorm(1,10))

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (10s sec)', ha='right', x=1)
plt.ylabel('trapEmax', ha='right', y=1)
plt.title(f'Energy vs time, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# rate stability
tlo, thi, tpb = 0, 1800, 1
nbx = int((thi-tlo)/tpb)

bl_hist, bins = np.histogram(df_hit['ts_sec'], bins=nbx,
                range=[tlo, thi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

# bl_cut_lo, bl_cut_hi = 8500,10000

# plt.axvline(bl_cut_lo, c='r', lw=1)
# plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('timestamp (10s seconds)', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.title('Rate vs Time')
plt.show()

In [ ]:
# use baseline cut and get 1460 events
elo, ehi = 3800, 4500
df_cut = df_hit.query('bl > 8500 and bl < 10000').copy()
e_cut = df_cut.query(f' trapEmax > {elo} and trapEmax < {ehi}').copy()

#create dcr/E 
df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

#creat new DCR
const = 0.0555
df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

# #create 10-90
# df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

# #create 50-90
# df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

#create 0-50
df_cut['tp0_50'] = df_cut['tp_50']- df_cut['tp_0']

In [ ]:
# Baseline vs E

elo, ehi, epb = 3950, 4025, 1
tlo, thi, tpb = 0, 1800, 30
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(e_cut['ts_sec'], e_cut['trapEmax'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (s)', ha='right', x=1)
plt.ylabel('trapEmax', ha='right', y=1)
plt.title(f'Energy vs time, 1460 peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# E vs t

elo, ehi, epb = 300, 400, 1
tlo, thi, tpb = 0, 1800, 30
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_hit['ts_sec'], df_hit['trapEmax_cal'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='viridis', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (s)', ha='right', x=1)
plt.ylabel('trapEmax_cal', ha='right', y=1)
plt.title(f'Energy vs time, 352 keV peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E

elo, ehi, epb = 1440, 1480, 0.5
tlo, thi, tpb = 0, 1800, 30
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(e_cut['ts_sec'], e_cut['trapEmax_cal'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='viridis', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (s)', ha='right', x=1)
plt.ylabel('trapEmax', ha='right', y=1)
plt.title(f'Energy vs time, 1460 peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E

# elo, ehi, epb = 1440, 1480, 0.5
elo, ehi, epb = 340, 360, 1
tlo, thi, tpb = 0, 1800, 30
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_cut['ts_sec'], df_cut['trapEmax_cal'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='viridis', norm=LogNorm(10, 40))

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (s)', ha='right', x=1)
plt.ylabel('trapEmax_cal', ha='right', y=1)
plt.title(f'Energy vs time, 352 peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E

elo, ehi, epb = 3950, 4025, 1
tlo, thi, tpb = 0, 1800, 1
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_cut['ts_sec'], df_cut['trapEftp'], bins=[nbx,nby],
                       range=[[tlo, thi], [elo, ehi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('timestamp (s)', ha='right', x=1)
plt.ylabel('trapEmax', ha='right', y=1)
plt.title(f'Energy vs time, 1460 peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Baseline vs E

elo, ehi, epb = 3950, 4025, 1
tlo, thi, tpb = 0, 1800, 1
nbx = int((thi-tlo)/tpb)
nby = int((ehi-elo)/epb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['trapEftp'], bins=[nbx,nbx],
                       range=[[elo, ehi], [elo, ehi]], cmap='jet', norm=LogNorm())

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('trapEftp', ha='right', y=1)
plt.title(f'Energy vs time, 1460 peak, {runtype} run {run}')
# plt.tight_layout()
plt.show()

In [ ]:
# Make energy spectrum
# elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 8000, 10
elo, ehi, epb = 3800, 4100, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])
energy_rt = np.divide(energy_hist, rt_min * 60)

plt.semilogy(bins[1:], energy_rt, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: r = {radius} mm; theta = {angle_det} deg\n{runtype} run {run}, {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# Make energy spectrum
# elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 8000, 10
elo, ehi, epb = 3800, 4100, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEftp'], bins=nbx,
                range=[elo, ehi])
energy_rt = np.divide(energy_hist, rt_min * 60)

plt.semilogy(bins[1:], energy_rt, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: r = {radius} mm; theta = {angle_det} deg\n{runtype} run {run}, {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()